Import required Libraries

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.1 MB/s eta 0:00:00


In [2]:
!pip install torch torchvision

In [3]:
import torch
print(torch.__version__)

2.0.1+cu118


In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
from sklearn.model_selection import train_test_split
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch import optim

# Load and Preprocess Data

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load the dataset

In [8]:
buzzfeed_fake_content = pd.read_csv("/content/drive/My Drive/Fake_Real_news_dataset/BuzzFeed_fake_news_content.csv")
buzzfeed_real_content = pd.read_csv("/content/drive/My Drive/Fake_Real_news_dataset/BuzzFeed_real_news_content.csv")
politifact_fake_content = pd.read_csv("/content/drive/My Drive/Fake_Real_news_dataset/PolitiFact_fake_news_content.csv")
politifact_real_content = pd.read_csv("/content/drive/My Drive/Fake_Real_news_dataset//PolitiFact_real_news_content.csv")

Create labels (0 for fake, 1 for real)

In [9]:
buzzfeed_fake_content['label']=0
buzzfeed_real_content['label']=1
politifact_fake_content['label']=0
politifact_real_content['label']=1

In [10]:
buzzfeed_fake_content.head()

,id,title,text,url,top_img,authors,source,publish_date,movies,images,canonical_link,meta_data,label
0,Fake_1-Webpage,Proof The Mainstream Media Is Manipulating The...,I woke up this morning to find a variation of ...,http://www.addictinginfo.org/2016/09/19/proof-...,http://addictinginfo.addictinginfoent.netdna-c...,Wendy Gittleson,http://www.addictinginfo.org,{'$date': 1474243200000},NaN,"http://i.imgur.com/JeqZLhj.png,http://addictin...",http://addictinginfo.com/2016/09/19/proof-the-...,"{""publisher"": ""Addicting Info | The Knowledge ...",0
1,Fake_10-Webpage,Charity: Clinton Foundation Distributed “Water...,Former President Bill Clinton and his Clinton ...,http://eaglerising.com/36899/charity-clinton-f...,http://eaglerising.com/wp-content/uploads/2016...,View All Posts,http://eaglerising.com,{'$date': 1474416521000},NaN,http://constitution.com/wp-content/uploads/201...,http://eaglerising.com/36899/charity-clinton-f...,"{""description"": ""The possibility that CHAI dis...",0
2,Fake_11-Webpage,A Hillary Clinton Administration May be Entire...,After collapsing just before trying to step in...,http://eaglerising.com/36880/a-hillary-clinton...,http://eaglerising.com/wp-content/uploads/2016...,"View All Posts,Tony Elliott",http://eaglerising.com,{'$date': 1474416638000},NaN,http://constitution.com/wp-content/uploads/201...,http://eaglerising.com/36880/a-hillary-clinton...,"{""description"": ""Hillary Clinton may be the fi...",0
3,Fake_12-Webpage,Trump’s Latest Campaign Promise May Be His Mos...,"Donald Trump is, well, deplorable. He’s sugges...",http://www.addictinginfo.org/2016/09/19/trumps...,http://addictinginfo.addictinginfoent.netdna-c...,John Prager,http://www.addictinginfo.org,{'$date': 1474243200000},NaN,"http://i.imgur.com/JeqZLhj.png,http://2.gravat...",http://addictinginfo.com/2016/09/19/trumps-lat...,"{""publisher"": ""Addicting Info | The Knowledge ...",0
4,Fake_13-Webpage,Website is Down For Maintenance,Website is Down For Maintenance,http://www.proudcons.com/clinton-foundation-ca...,NaN,NaN,http://www.proudcons.com,NaN,NaN,NaN,NaN,"{""og"": {""url"": ""http://www.proudcons.com"", ""ty...",0


Combine real and fake news content

In [39]:
news_data = pd.concat([politifact_fake_content,
                       buzzfeed_fake_content, politifact_real_content,buzzfeed_real_content
                      ])

In [40]:
news_data.shape

(422, 13)

In [12]:
news_data.head()

,id,title,text,url,top_img,authors,source,publish_date,movies,images,canonical_link,meta_data,label
0,Real_1-Webpage,Trump Just Insulted Millions Who Lost Everythi...,16.8k SHARES SHARE THIS STORY\n\nHillary Clint...,http://occupydemocrats.com/2016/09/27/trump-ju...,http://occupydemocrats.com/wp-content/uploads/...,"Brett Bose,Grant Stern,Steve Bernstein,Natalie...",http://occupydemocrats.com,{'$date': 1474934400000},NaN,http://occupydemocrats.com/wp-content/uploads/...,http://occupydemocrats.com/2016/09/27/trump-ju...,"{""generator"": ""Powered by Visual Composer - dr...",0
1,Real_10-Webpage,Famous dog killed in spot she waited a year fo...,Famous dog killed in spot she waited a year fo...,http://rightwingnews.com/top-news/famous-dog-k...,http://rightwingnews.com/wp-content/uploads/20...,NaN,http://rightwingnews.com,{'$date': 1474948336000},NaN,http://rightwingnews.com/wp-content/uploads/20...,http://rightwingnews.com/top-news/famous-dog-k...,"{""googlebot"": ""noimageindex"", ""og"": {""site_nam...",0
2,Real_100-Webpage,House oversight panel votes Clinton IT chief i...,Story highlights The House Oversight panel vot...,http://cnn.it/2deaH2d,http://i2.cdn.cnn.com/cnnnext/dam/assets/16091...,"Tom Lobianco,Deirdre Walsh",http://cnn.it,NaN,NaN,http://i2.cdn.cnn.com/cnnnext/dam/assets/17050...,http://www.cnn.com/2016/09/22/politics/bryan-p...,"{""description"": ""Members of the House Oversigh...",0
3,Real_101-Webpage,America Just Tragically Lost A Country Music I...,We are absolutely heartbroken to hear about th...,http://newsbake.com/entertainment-news/music-e...,http://newsbake.com/wp-content/uploads/2016/05...,Nancy Wells,http://newsbake.com,{'$date': 1474898600000},https://www.youtube.com/embed/8ozTJcu-_BU,http://0.gravatar.com/avatar/0d702c6042933cd78...,http://newsbake.com/entertainment-news/music-e...,"{""shareaholic"": {""site_name"": ""NewsBake"", ""lan...",0
4,Real_102-Webpage,Monuments to the Battle for the New South,"Nine years ago, a driver lost control of his p...",http://politi.co/2dd9U1x,http://static.politico.com/25/ed/85332de14c45b...,"Jack Shafer,Lisa Rab",http://politi.co,{'$date': 1473941820000},NaN,http://static.politico.com/25/ed/85332de14c45b...,http://www.politico.com/magazine/story/2016/09...,"{""description"": ""Virginia, increasingly divers...",0


Split data into train and test sets

In [41]:
labels = news_data['label']
labels

0     0
1     0
2     0
3     0
4     0
     ..
86    1
87    1
88    1
89    1
90    1
Name: label, Length: 422, dtype: int64

In [42]:
print(labels.to_list())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [43]:
news_data['combined_text'] = news_data.title + ' ' + news_data.text

In [44]:
text_data = news_data['combined_text'].to_list()

In [18]:
print(len(text_data))

422


Cleaning data

In [45]:
data_text_lower = []
for i in range(len(text_data)):
  data_text_lower.append(text_data[i].lower())
data_text_lower[10]

'ny gov. andrew cuomo disputes trump\'s claim cops are \'afraid\' to do their jobs story highlights "i don\'t know how you could have been more aggressive than we were here," cuomo said\n\nahmad khan rahami was captured monday after a manhunt and shootout\n\nwashington (cnn) new york gov. andrew cuomo said tuesday that republican presidential candidate donald trump is wrong to suggest that the new york bombing suspect shouldn\'t receive medical care.\n\n"i understand the anger that donald trump is speaking to (but) this is america and this is our system and you are innocent until proven guilty and you have a right to counsel and that is the constitution of the united states of america," he said on cnn\'s "new day."\n\n"and that\'s what makes us who we are. that\'s what makes us special. and if you give that up, alisyn, then you have defeated yourself," cuomo, a democrat who has endorsed hillary clinton for president this cycle, told cnn\'s alisyn camerota. "that is the code of democrac

In [46]:
datanew = list(data_text_lower)

In [47]:
import nltk
from nltk.corpus import stopwords


# Download the NLTK stopwords dataset if you haven't already
nltk.download('stopwords')

# Get the list of English stopwords
stop_words = set(stopwords.words('english'))

# Function to remove stopwords from a single sentence
def remove_stopwords(sentence):
    words = sentence.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Remove stopwords from each sentence in the dataset
filtered_data = [remove_stopwords(sentence) for sentence in datanew]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
print(len(filtered_data))

422


In [49]:
len(labels)

422

In [50]:
X_train, X_test, y_train, y_test = train_test_split( filtered_data,labels, test_size=0.20)

# Tokenization and Data Loading, Model Building, Training, and Evaluation

In [51]:
# Load the pre-trained BERT model
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 Tokenize and convert to tensors

In [52]:
def tokenize_text(text, max_length):
    return tokenizer(text, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')

max_length = 128
train_texts = list(X_train)
test_texts = list(X_test)

train_encodings = tokenize_text(train_texts, max_length)
test_encodings = tokenize_text(test_texts, max_length)

train_labels = torch.tensor(list(y_train))
test_labels = torch.tensor(list(y_test))

In [27]:
train_encodings.input_ids

tensor([[    0,  9981, 20985,  ...,  1538,   295,     2],
        [    0, 45397,    35,  ...,   400,  1986,     2],
        [    0, 10536,    35,  ...,   627,  2369,     2],
        ...,
        [    0, 24139, 20125,  ...,  1846,  2021,     2],
        [    0, 31372, 11721,  ...,     4,   951,     2],
        [    0, 38060,   637,  ...,     4,    67,     2]])

Loss and optimizer

In [53]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-05)

Training

In [54]:
# Define hyperparameters
num_epochs = 5
batch_size = 64

# Create DataLoader for training data
train_dataset = torch.utils.data.TensorDataset(train_encodings.input_ids, train_encodings.attention_mask, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Check and set device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move the model to the selected device

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0.0  # Initialize total loss for this epoch

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        # Compute loss
        loss = outputs.loss
        total_loss += loss.item()  # Accumulate the loss for this batch

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Calculate the average loss for this epoch
    average_loss = total_loss / len(train_loader)

    # Print epoch-wise information
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}')
print("Training complete.")

Epoch [1/5], Loss: 0.7038
Epoch [2/5], Loss: 0.6941
Epoch [3/5], Loss: 0.6851
Epoch [4/5], Loss: 0.6888
Epoch [5/5], Loss: 0.6834
Training complete.


Evaluation

In [55]:
model.eval()
with torch.no_grad():
    test_input_ids = test_encodings.input_ids.to(device)
    test_attention_mask = test_encodings.attention_mask.to(device)
    test_labels = test_labels.to(device)
    test_outputs = model(test_input_ids, attention_mask=test_attention_mask)
    test_preds = test_outputs.logits.argmax(dim=1)

acc = accuracy_score(test_labels.cpu().numpy(), test_preds.cpu().numpy())
precision = precision_score(test_labels.cpu().numpy(), test_preds.cpu().numpy(),zero_division=1)
recall = recall_score(test_labels.cpu().numpy(), test_preds.cpu().numpy(),zero_division=1)
f1 = f1_score(test_labels.cpu().numpy(), test_preds.cpu().numpy(),zero_division=1)

print(f"Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

Accuracy: 0.5412, Precision: 0.5067, Recall: 0.9500, F1-Score: 0.6609
